
<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: orange; background-color: #ffffff;">Ventilator Pressure Prediction</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;">Interactive Clustering On U_IN and PRESSURE</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

This notebook is heavily inspired by [**Chris Deotte's notebook**](https://www.kaggle.com/cdeotte/rapids-knn-and-kmeans-0-600). Please check it out and give it an upvote. I simply wanted to learn all I code and I did that by refactoring some of the code and implementing it using Plotly.

---

**Addendum:**

I wanted to see how different the U_IN and R&C values could be given a similar pressure plot. As such I decided to cluster the time series by using Pressure



<br>



<p id="toc"></p>

<br><br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: darkorange; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#clustering">4&nbsp;&nbsp;&nbsp;&nbsp;CLUSTERING WITH PLOTLY</a></h3>

---

<br>

<a id="imports"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: darkorange;" id="imports">0&nbsp;&nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In [ ]:
print("\n... IMPORTS STARTING ...\n")

!pip install weightedstats
import weightedstats as ws

print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.model_selection import GroupKFold;

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: darkorange; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">PRIMARY TASK DESCRIPTION</b>


**In this notebook, we will use RAPIDS Nearest Neighbors and RAPIDS KMeans to help us understand the data. We will find similar u_in time series and display them. We will notice that many time series with the same u_in have different pressure! This will help us to understand how parameters R and C affect pressure when the same u_in is used for different R and C combinations.**

---

<br>

<br>


<a id="setup"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: darkorange; background-color: #ffffff;"  id="setup">2&nbsp;&nbsp;SETUP&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">2.1 ACCELERATOR DETECTION</h3>

---

In this cell we will ascertain the training strategy. I am setting this notebook up so that it can perform three fundamental operations. 
* Clustering (**GPU**)
* Neural Network Training (**TPU**)
* Submission (**CPU**)

---

In order to use **`TPU`**, we use **`TPUClusterResolver`** for the initialization which is necessary to connect to the remote cluster and initialize cloud TPUs. Let's go over two important points

1. When using TPU on Kaggle, you don't need to specify arguments for **`TPUClusterResolver`**
2. However, on **G**oogle **C**ompute **E**ngine (**GCE**), you will need to do the following:

<br>

```python
# The name you gave to the TPU to use
TPU_WORKER = 'my-tpu-name'

# or you can also specify the grpc path directly
# TPU_WORKER = 'grpc://xxx.xxx.xxx.xxx:8470'

# The zone you chose when you created the TPU to use on GCP.
ZONE = 'us-east1-b'

# The name of the GCP project where you created the TPU to use on GCP.
PROJECT = 'my-tpu-project'

tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER, zone=ZONE, project=PROJECT)
```

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- Although the Tensorflow documentation says it is the <b>project name</b> that should be provided for the argument <b><code>`project`</code></b>, it is actually the <b>Project ID</b>, that you should provide. This can be found on the GCP project dashboard page.<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCES:</b><br><br>
    - <a href="https://www.tensorflow.org/guide/tpu#tpu_initialization"><b>Guide - Use TPUs</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/distribute/cluster_resolver/TPUClusterResolver"><b>Doc - TPUClusterResolver</b></a><br>

</div>

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()} ...\n")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy()     
    if tf.config.experimental.list_physical_devices('GPU'):
        print(f"\n ... RUNNING ON GPU ...\n")
        import cudf, cuml, cupy
        from numba import cuda
        from cuml.neighbors import NearestNeighbors
    else:
        print(f"\n ... RUNNING ON CPU ...\n")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">2.2 COMPETITION DATA ACCESS</h3>

---

TPUs read data must be read directly from **G**oogle **C**loud **S**torage **(GCS)**. Kaggle provides a utility library – **`KaggleDatasets`** – which has a utility function **`.get_gcs_path`** that will allow us to access the location of our input datasets within **GCS**.<br><br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; TIPS:</b><br><br>- If you have multiple datasets attached to the notebook, you should pass the name of a specific dataset to the <b><code>`get_gcs_path()`</code></b> function. <i>In our case, the name of the dataset is the name of the directory the dataset is mounted within.</i><br><br>
</div>

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('ventilator-pressure-prediction')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/ventilator-pressure-prediction"
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

    
print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">2.3 BASIC DATA DEFINITIONS & INITIALIZATIONS</h3>

---


In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

# If set to false, pre-generated dataframes will be loaded from disk using pandas
DO_CLUSTERING=True

print("\n... TRAIN DATAFRAME ..\n")
if DO_CLUSTERING:
    TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
    train_df = cudf.read_csv(TRAIN_CSV)
else:
    TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
    train_df = pd.read_csv(TRAIN_CSV)
display(train_df)

print("\n... TEST DATAFRAME ..\n")
if DO_CLUSTERING:
    TEST_CSV = os.path.join(DATA_DIR, "test.csv")
    test_df = cudf.read_csv(TEST_CSV)
else:
    TEST_CSV = os.path.join(DATA_DIR, "test.csv")
    test_df = pd.read_csv(TEST_CSV)
display(test_df)

print("\n... SAMPLE SUBMISSION DATAFRAME ..\n")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)
display(ss_df)


# Get some basic breath information
N_TRAIN_BREATHS = len(train_df.groupby("breath_id").count())
N_TEST_BREATHS = len(test_df.breath_id.value_counts())
ROWS_PER_BREATH = 80

# Add in a piece of basic information (which step in each breath's time series are we?)
test_df["breath_step"] = np.arange(ROWS_PER_BREATH).tolist()*(len(test_df)//ROWS_PER_BREATH)
train_df["breath_step"] = np.arange(ROWS_PER_BREATH).tolist()*(len(train_df)//ROWS_PER_BREATH)

# Get some basic information around pressures
POSSIBLE_PRESSURES = train_df.pressure.unique().sort_values().values
PRESSURE_DELTA_STEP = float((POSSIBLE_PRESSURES[1:]-POSSIBLE_PRESSURES[:-1]).mean())

print("\n\n... BASIC DATA SETUP FINISHING ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">2.4 LEVERAGING XLA OPTIMIZATIONS</h3>

---


**XLA** (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes. **The results are improvements in speed and memory usage**.

<br>

When a TensorFlow program is run, all of the operations are executed individually by the TensorFlow executor. Each TensorFlow operation has a precompiled GPU/TPU kernel implementation that the executor dispatches to.

XLA provides us with an alternative mode of running models: it compiles the TensorFlow graph into a sequence of computation kernels generated specifically for the given model. Because these kernels are unique to the model, they can exploit model-specific information for optimization.<br><br>

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- XLA can not currently compile functions where dimensions are not inferrable: that is, if it's not possible to infer the dimensions of all tensors without running the entire computation<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- XLA compilation is only applied to code that is compiled into a graph (in <b>TF2</b> that's only a code inside <b><code>tf.function</code></b>).<br>- The <b><code>jit_compile</code></b> API has must-compile semantics, i.e. either the entire function is compiled with XLA, or an <b><code>errors.InvalidArgumentError</code></b> exception is thrown)
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>    - <a href="https://www.tensorflow.org/xla"><b>XLA: Optimizing Compiler for Machine Learning</b></a><br>
</div>

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

<br>


<a id="helper_functions"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: darkorange; background-color: #ffffff;" id="helper_functions">
    3&nbsp;&nbsp;HELPER FUNCTION & CLASSES&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">3.1 GENERAL HELPER FUNCTIONS</h3>

---

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def compute_weighted_median(_values, _weights, redux_factor=1):
    """ Compute a weighted median using knn distances
    
    Args:
        values (): TBD
        weights (): TBD
    
    Returns:
        The weighted median
    """
    return np.median(flatten_l_o_l([[_v,]*int(_w//redux_factor) for _v,_w in zip(_values,_weights)]))


def add_features(df, 
                 U_IN_N_FORWARD=3,
                 U_IN_N_BACKWARD=3,
                 U_OUT_N_FORWARD=1, 
                 U_OUT_N_BACKWARD=1, 
                 use_rc=True):
    """ TBD """
    
    # Features
    print("\n... Add general features ...\n")
    df['uin_auc'] = df['time_step'] * df['u_in']
    df['uin_auc'] = df.groupby('breath_id')['uin_auc'].cumsum()
    df['uin_csum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['cross3']= df['time_step']*df['u_in']
    df['cross3_sqd_1']= df['time_step']*df['u_in']**2
    df['cross3_sqd_2']= df['time_step']**2*df['u_in']
    df['cross3_cubed_1']= df['time_step']*df['u_in']**3
    df['cross3_cubed_2']= df['time_step']**3*df['u_in']

    
    # Uin Lag
    print("\t... Add lag and advance UIN features ...")
    for i in range(1, U_IN_N_BACKWARD+1):
        df[f'u_in_{i}_back'] = df.groupby("breath_id")['u_in'].shift(i).fillna(0)
    for i in range(1, U_IN_N_FORWARD+1):
        df[f'u_in_{i}_forw'] = df.groupby("breath_id")['u_in'].shift(-i).fillna(0)
    
    print("\t... Add lag and advance UOUT features ...")
    for i in range(1, U_OUT_N_BACKWARD+1):
        df[f'u_out_{i}_back'] = df.groupby("breath_id")['u_out'].shift(i).fillna(0)
    for i in range(1, U_OUT_N_FORWARD+1):
        df[f'u_out_{i}_forw'] = df.groupby("breath_id")['u_out'].shift(-i).fillna(0)
    
    print("\t... Add UIN and UOUT `diff` features ...")
    for i in range(1, U_IN_N_BACKWARD+1):
        df[f'u_in_diff_{i}_back'] = df['u_in'] - df[f'u_in_{i}_back']
    for i in range(1, U_OUT_N_BACKWARD+1):
        df[f'u_out_diff_{i}_back'] = df['u_out'] - df[f'u_out_{i}_back']
    for i in range(1, U_IN_N_FORWARD+1):
        df[f'u_in_diff_{i}_back'] = df['u_in'] - df[f'u_in_{i}_forw']
    for i in range(1, U_OUT_N_FORWARD+1):
        df[f'u_out_diff_{i}_forw'] = df['u_out'] - df[f'u_out_{i}_forw']
    
    print("\t... Add categorical features ...")
    if use_rc:
        # df['R_C'] = df['R'].astype(str)+"_"+df['C'].astype(str)
        df['_R'] = df['R'].astype(str)
        df['_C'] = df['C'].astype(str)
        df['R'] = df['R']/50
        df['C'] = df['C']/50
        df = pd.get_dummies(df,)
    
    print("\t... Reset dtypes for lower memory usage ...")
    for c in df.columns:
        if c in ["u_out", "breath_step"]:
            df[c] = df[c].astype("uint8")
        elif df[c].dtype=="float64":
            df[c] = df[c].astype("float32")
    
    # Final cleanup
    gc.collect(); gc.collect();
    
    return df

<br>


<a id="clustering"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: darkorange; background-color: #ffffff;" id="clustering">
    4&nbsp;&nbsp;CLUSTERING WITH PLOTLY&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transforam: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">4.1 DETERMINE SUBCATEGORIES OF DATA TO CLUSTER WITHIN</h3>

---

We will predivide the data into 9 seperate sets. Each set is representative of a combination of R and C values.<br>**I would recommend Chris Deotte's wonderful notebook for more information.**

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transforam: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">4.2 RESTRUCTURE (COMPRESS) THE ORIGINAL DATAFRAME STRUCTURE AND ADD IN SOME RELEVANT ADDITIONAL INFO</h3>

---

In [ ]:
print("\n... ADDING FEATURES TO TRAIN DATAFRAME ...\n")
train_df = add_features(train_df.to_pandas(), use_rc=True)
train_df = cudf.from_pandas(train_df)

print("\n... ADDING FEATURES TO TEST DATAFRAME ...\n")
test_df = add_features(test_df.to_pandas(), use_rc=True)
test_df = cudf.from_pandas(test_df)

display(train_df.head(3))
display(test_df.head(3))

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transforam: none; letter-spacing: 2px; color: darkorange; background-color: #ffffff;">4.5 RECREATE BREATHS W/ RAPIDS AND KNN</h3>

---

We will cluster breaths (from similar RC configs) together based on **`U_IN`** and other features. We will then blend these Nearest Neighbors to recreate the original **`PRESSURE`** curve and see how we do...

<br>

In [ ]:
test_df = test_df.to_pandas()
test_df["breath_duration"] = test_df.groupby('breath_id')[['time_step']].transform('max')
test_df["exhale_steps"] = test_df.groupby('breath_id')[['u_out']].transform('sum')
test_df["inhale_steps"] = 80-test_df.groupby('breath_id')[['u_out']].transform('sum')
test_df = cudf.from_pandas(test_df)

train_df = train_df.to_pandas()
train_df["breath_duration"] = train_df.groupby('breath_id')[['time_step']].transform('max')
train_df["exhale_steps"] = train_df.groupby('breath_id')[['u_out']].transform('sum')
train_df["inhale_steps"] = 80-train_df.groupby('breath_id')[['u_out']].transform('sum')
train_df = cudf.from_pandas(train_df)

display(train_df)
display(test_df)

In [ ]:
RS = cuml.preprocessing.RobustScaler()
FEATURE_COLS = [_c for _c in train_df.columns if _c not in ['id', 'breath_id', 'pressure']]
train_df[FEATURE_COLS] = RS.fit_transform(train_df[FEATURE_COLS])
test_df[FEATURE_COLS] = RS.transform(test_df[FEATURE_COLS])

display(train_df)
display(test_df)

In [ ]:
def compress_df(df):
    cdf = df.groupby('breath_id').collect().reset_index()
    
    flatten_cols = list(set([_c for _c in cdf.columns if "_back" in _c]+\
                   [_c for _c in cdf.columns if "_forw" in _c]+\
                   [_c for _c in cdf.columns if "cross" in _c]+\
                   [_c for _c in cdf.columns if "u_in" in _c]+\
                   [_c for _c in cdf.columns if "u_out" in _c]+\
                   [_c for _c in cdf.columns if "uin" in _c]+\
                   [_c for _c in cdf.columns if "uout" in _c]+\
                   ["breath_step", "time_step"]))
    
    for j, _c in enumerate(flatten_cols):
        for i in range(ROWS_PER_BREATH): cdf[f'{chr(97+j)}_{i}'] = cdf[_c].list.get(i)
    
    if "pressure" in cdf.columns:
        for i in range(ROWS_PER_BREATH): cdf[f'z_{i}'] = cdf["pressure"].list.get(i)
        flatten_cols.append("pressure")
    cdf.drop(columns=flatten_cols+["id",], inplace=True)
    
    # Get 
    REPEAT_COLS = list(set([_c for _c in cdf.columns if "R" in _c]+\
                   [_c for _c in cdf.columns if "C" in _c]+\
                   ["breath_duration", "exhale_steps", "inshale_steps"]))
    REPEAT_COLS = ["R", "C", "_R_20", "_R_5", "_R_50", "_C_10", "_C_20", "_C_50", "breath_duration", "exhale_steps", "inhale_steps"]
    for _rc in REPEAT_COLS:
        cdf[_rc] = cdf[_rc].list.get(0)
    return cdf

train_df = compress_df(train_df)
test_df = compress_df(test_df)

display(train_df)
display(test_df)

In [ ]:
PRESSURE_COLS = [f"z_{i}" for i in range(80)]
USE_COLS = [_c for _c in train_df.columns if _c not in ["breath_id", "id"]+[f"z_{i}" for i in range(80)]]
BLEND_NEIGHBORS = 100
RESTRICT_RC=True

In [ ]:
# Explore For Breath 1
BREATH_ID=1
gt_breath = train_df.loc[train_df.breath_id==BREATH_ID].to_pandas()
BREATH_R = gt_breath.R.values[0]
BREATH_C = gt_breath.C.values[0]

df_to_use = train_df.copy()[(train_df.R==BREATH_R)&(train_df.C==BREATH_C)].reset_index(drop=True) if RESTRICT_RC else train_df.copy()

# Fit the model
model = NearestNeighbors(n_neighbors=BLEND_NEIGHBORS, metric="l1",)
model.fit(df_to_use[USE_COLS])

distances, indices = model.kneighbors(gt_breath[USE_COLS])
distances, indices = distances[0, 1:], indices[0, 1:] #discard GT

gt_breath_pressure = gt_breath[PRESSURE_COLS].squeeze().values
nn_pressure_df = df_to_use.iloc[indices][PRESSURE_COLS]
nn_pressure_df.insert(loc=0, name="nn_distance", value=distances)
nn_pressures = nn_pressure_df[PRESSURE_COLS].values

for UP_TO in [BLEND_NEIGHBORS-1, 75, 50, 25, 12, 5, 2, 1]:
    print("\n\n\n")
    plt.figure(figsize=(20, 8))
    for i in range(UP_TO):
        if i==UP_TO-1:
            plt.plot(nn_pressures[i].get(), color="lightblue", linewidth=1, label="NEIGHBORS")
        else:
            plt.plot(nn_pressures[i].get(), color="lightblue")
    plt.plot(gt_breath_pressure, color="blue", linewidth=3, label="GROUND TRUTH")
    
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    mean_mae = np.abs(gt_breath_pressure-mean_pressure).mean()
    plt.plot(mean_pressure, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    median_mae = np.abs(gt_breath_pressure-median_pressure).mean()
    plt.plot(median_pressure, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
    
    plt.title(f"{UP_TO} NEAREST NEIGHBORS VS GROUND TRUTH\n\nMEAN MAE = {mean_mae:.5f}\nMEDIAN MAE = {median_mae:.5f}", fontweight="bold")
    plt.xlabel("Step In Breath", fontweight="bold")
    plt.ylabel("Pressure (mm/H2O)", fontweight="bold")

    plt.legend(loc="upper right", prop={"size":14})
    plt.grid(which="both")
    
    plt.tight_layout()
    plt.show()
    
neighbor_means = []
neighbor_medians = []
for UP_TO in tqdm(range(1, BLEND_NEIGHBORS), total=BLEND_NEIGHBORS-1):
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    neighbor_medians.append(np.abs(gt_breath_pressure-median_pressure).mean())
    
    
    
# ERROR PLOT - WHICH NUMBER OF NEIGHBORS IS BEST
plt.figure(figsize=(20, 8))

MIN_MAE = min(min(neighbor_means), min(neighbor_medians))

plt.plot(neighbor_means, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_means, y2=MIN_MAE, color="deepskyblue", alpha=0.25)

plt.plot(neighbor_medians, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_medians, y2=MIN_MAE, color="mediumspringgreen", alpha=0.25)

IDEAL_NUMBER_OF_NEIGHBORS = np.argmin(neighbor_medians) if min(neighbor_means)>min(neighbor_medians) else np.argmin(neighbor_means)
BLEND_STYLE = "MEDIAN" if min(neighbor_means)>min(neighbor_medians) else "MEAN"
plt.axvline(x=IDEAL_NUMBER_OF_NEIGHBORS, color='blue', linestyle="dotted")

plt.xlabel("Number of Neighbors", fontweight="bold")
plt.ylabel("MAE When Compared to Ground Truth", fontweight="bold")
plt.title(f"NEAREST NEIGHBOR MEAN v. MEDIAN FOR DIFFERENT AMOUNTS OF NEIGHBORS", fontweight="bold")
plt.legend(loc="upper right", prop={"size":14})
plt.grid(which="both")

plt.tight_layout()
plt.show()

print(f"\n\n... THE IDEAL NUMBER OF NEIGHBORS IS {IDEAL_NUMBER_OF_NEIGHBORS}, USING `{BLEND_STYLE}` WITH AN MAE OF {MIN_MAE} ...\n\n")

In [ ]:
# Explore For Breath 28
BREATH_ID=28

gt_breath = train_df.loc[train_df.breath_id==BREATH_ID].to_pandas()
BREATH_R = gt_breath.R.values[0]
BREATH_C = gt_breath.C.values[0]

df_to_use = train_df.copy()[(train_df.R==BREATH_R)&(train_df.C==BREATH_C)].reset_index(drop=True) if RESTRICT_RC else train_df.copy()

# Fit the model
model = NearestNeighbors(n_neighbors=BLEND_NEIGHBORS, metric="l1",)
model.fit(df_to_use[USE_COLS])

distances, indices = model.kneighbors(gt_breath[USE_COLS])
distances, indices = distances[0, 1:], indices[0, 1:] #discard GT

gt_breath_pressure = gt_breath[PRESSURE_COLS].squeeze().values
nn_pressure_df = df_to_use.iloc[indices][PRESSURE_COLS]
nn_pressure_df.insert(loc=0, name="nn_distance", value=distances)
nn_pressures = nn_pressure_df[PRESSURE_COLS].values

for UP_TO in [BLEND_NEIGHBORS-1, 75, 50, 25, 12, 5, 2, 1]:
    print("\n\n\n")
    plt.figure(figsize=(20, 8))
    for i in range(UP_TO):
        if i==UP_TO-1:
            plt.plot(nn_pressures[i].get(), color="lightblue", linewidth=1, label="NEIGHBORS")
        else:
            plt.plot(nn_pressures[i].get(), color="lightblue")
    plt.plot(gt_breath_pressure, color="blue", linewidth=3, label="GROUND TRUTH")
    
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    mean_mae = np.abs(gt_breath_pressure-mean_pressure).mean()
    plt.plot(mean_pressure, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    median_mae = np.abs(gt_breath_pressure-median_pressure).mean()
    plt.plot(median_pressure, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
    
    plt.title(f"{UP_TO} NEAREST NEIGHBORS VS GROUND TRUTH\n\nMEAN MAE = {mean_mae:.5f}\nMEDIAN MAE = {median_mae:.5f}", fontweight="bold")
    plt.xlabel("Step In Breath", fontweight="bold")
    plt.ylabel("Pressure (mm/H2O)", fontweight="bold")

    plt.legend(loc="upper right", prop={"size":14})
    plt.grid(which="both")
    
    plt.tight_layout()
    plt.show()
    
neighbor_means = []
neighbor_medians = []
for UP_TO in tqdm(range(1, BLEND_NEIGHBORS), total=BLEND_NEIGHBORS-1):
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    neighbor_medians.append(np.abs(gt_breath_pressure-median_pressure).mean())
    
    
    
# ERROR PLOT - WHICH NUMBER OF NEIGHBORS IS BEST
plt.figure(figsize=(20, 8))

MIN_MAE = min(min(neighbor_means), min(neighbor_medians))

plt.plot(neighbor_means, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_means, y2=MIN_MAE, color="deepskyblue", alpha=0.25)

plt.plot(neighbor_medians, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_medians, y2=MIN_MAE, color="mediumspringgreen", alpha=0.25)

IDEAL_NUMBER_OF_NEIGHBORS = np.argmin(neighbor_medians) if min(neighbor_means)>min(neighbor_medians) else np.argmin(neighbor_means)
BLEND_STYLE = "MEDIAN" if min(neighbor_means)>min(neighbor_medians) else "MEAN"
plt.axvline(x=IDEAL_NUMBER_OF_NEIGHBORS, color='blue', linestyle="dotted")

plt.xlabel("Number of Neighbors", fontweight="bold")
plt.ylabel("MAE When Compared to Ground Truth", fontweight="bold")
plt.title(f"NEAREST NEIGHBOR MEAN v. MEDIAN FOR DIFFERENT AMOUNTS OF NEIGHBORS", fontweight="bold")
plt.legend(loc="upper right", prop={"size":14})
plt.grid(which="both")

plt.tight_layout()
plt.show()

print(f"\n\n... THE IDEAL NUMBER OF NEIGHBORS IS {IDEAL_NUMBER_OF_NEIGHBORS}, USING `{BLEND_STYLE}` WITH AN MAE OF {MIN_MAE} ...\n\n")

In [ ]:
# Explore For Breath 87
BREATH_ID=87

gt_breath = train_df.loc[train_df.breath_id==BREATH_ID].to_pandas()
BREATH_R = gt_breath.R.values[0]
BREATH_C = gt_breath.C.values[0]

df_to_use = train_df.copy()[(train_df.R==BREATH_R)&(train_df.C==BREATH_C)].reset_index(drop=True) if RESTRICT_RC else train_df.copy()

# Fit the model
model = NearestNeighbors(n_neighbors=BLEND_NEIGHBORS, metric="l1",)
model.fit(df_to_use[USE_COLS])

distances, indices = model.kneighbors(gt_breath[USE_COLS])
distances, indices = distances[0, 1:], indices[0, 1:] #discard GT

gt_breath_pressure = gt_breath[PRESSURE_COLS].squeeze().values
nn_pressure_df = df_to_use.iloc[indices][PRESSURE_COLS]
nn_pressure_df.insert(loc=0, name="nn_distance", value=distances)
nn_pressures = nn_pressure_df[PRESSURE_COLS].values

for UP_TO in [BLEND_NEIGHBORS-1, 75, 50, 25, 12, 5, 2, 1]:
    print("\n\n\n")
    plt.figure(figsize=(20, 8))
    for i in range(UP_TO):
        if i==UP_TO-1:
            plt.plot(nn_pressures[i].get(), color="lightblue", linewidth=1, label="NEIGHBORS")
        else:
            plt.plot(nn_pressures[i].get(), color="lightblue")
    plt.plot(gt_breath_pressure, color="blue", linewidth=3, label="GROUND TRUTH")
    
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    mean_mae = np.abs(gt_breath_pressure-mean_pressure).mean()
    plt.plot(mean_pressure, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    median_mae = np.abs(gt_breath_pressure-median_pressure).mean()
    plt.plot(median_pressure, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
    
    plt.title(f"{UP_TO} NEAREST NEIGHBORS VS GROUND TRUTH\n\nMEAN MAE = {mean_mae:.5f}\nMEDIAN MAE = {median_mae:.5f}", fontweight="bold")
    plt.xlabel("Step In Breath", fontweight="bold")
    plt.ylabel("Pressure (mm/H2O)", fontweight="bold")

    plt.legend(loc="upper right", prop={"size":14})
    plt.grid(which="both")
    
    plt.tight_layout()
    plt.show()
    
neighbor_means = []
neighbor_medians = []
for UP_TO in tqdm(range(1, BLEND_NEIGHBORS), total=BLEND_NEIGHBORS-1):
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    neighbor_medians.append(np.abs(gt_breath_pressure-median_pressure).mean())
    
    
    
# ERROR PLOT - WHICH NUMBER OF NEIGHBORS IS BEST
plt.figure(figsize=(20, 8))

MIN_MAE = min(min(neighbor_means), min(neighbor_medians))

plt.plot(neighbor_means, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_means, y2=MIN_MAE, color="deepskyblue", alpha=0.25)

plt.plot(neighbor_medians, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_medians, y2=MIN_MAE, color="mediumspringgreen", alpha=0.25)

IDEAL_NUMBER_OF_NEIGHBORS = np.argmin(neighbor_medians) if min(neighbor_means)>min(neighbor_medians) else np.argmin(neighbor_means)
BLEND_STYLE = "MEDIAN" if min(neighbor_means)>min(neighbor_medians) else "MEAN"
plt.axvline(x=IDEAL_NUMBER_OF_NEIGHBORS, color='blue', linestyle="dotted")

plt.xlabel("Number of Neighbors", fontweight="bold")
plt.ylabel("MAE When Compared to Ground Truth", fontweight="bold")
plt.title(f"NEAREST NEIGHBOR MEAN v. MEDIAN FOR DIFFERENT AMOUNTS OF NEIGHBORS", fontweight="bold")
plt.legend(loc="upper right", prop={"size":14})
plt.grid(which="both")

plt.tight_layout()
plt.show()

print(f"\n\n... THE IDEAL NUMBER OF NEIGHBORS IS {IDEAL_NUMBER_OF_NEIGHBORS}, USING `{BLEND_STYLE}` WITH AN MAE OF {MIN_MAE} ...\n\n")

In [ ]:
# Explore For Breath 101
BREATH_ID=101

gt_breath = train_df.loc[train_df.breath_id==BREATH_ID].to_pandas()
BREATH_R = gt_breath.R.values[0]
BREATH_C = gt_breath.C.values[0]

df_to_use = train_df.copy()[(train_df.R==BREATH_R)&(train_df.C==BREATH_C)].reset_index(drop=True) if RESTRICT_RC else train_df.copy()

# Fit the model
model = NearestNeighbors(n_neighbors=BLEND_NEIGHBORS, metric="l1",)
model.fit(df_to_use[USE_COLS])

distances, indices = model.kneighbors(gt_breath[USE_COLS])
distances, indices = distances[0, 1:], indices[0, 1:] #discard GT

gt_breath_pressure = gt_breath[PRESSURE_COLS].squeeze().values
nn_pressure_df = df_to_use.iloc[indices][PRESSURE_COLS]
nn_pressure_df.insert(loc=0, name="nn_distance", value=distances)
nn_pressures = nn_pressure_df[PRESSURE_COLS].values

for UP_TO in [BLEND_NEIGHBORS-1, 75, 50, 25, 12, 5, 2, 1]:
    print("\n\n\n")
    plt.figure(figsize=(20, 8))
    for i in range(UP_TO):
        if i==UP_TO-1:
            plt.plot(nn_pressures[i].get(), color="lightblue", linewidth=1, label="NEIGHBORS")
        else:
            plt.plot(nn_pressures[i].get(), color="lightblue")
    plt.plot(gt_breath_pressure, color="blue", linewidth=3, label="GROUND TRUTH")
    
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    mean_mae = np.abs(gt_breath_pressure-mean_pressure).mean()
    plt.plot(mean_pressure, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    median_mae = np.abs(gt_breath_pressure-median_pressure).mean()
    plt.plot(median_pressure, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
    
    plt.title(f"{UP_TO} NEAREST NEIGHBORS VS GROUND TRUTH\n\nMEAN MAE = {mean_mae:.5f}\nMEDIAN MAE = {median_mae:.5f}", fontweight="bold")
    plt.xlabel("Step In Breath", fontweight="bold")
    plt.ylabel("Pressure (mm/H2O)", fontweight="bold")

    plt.legend(loc="upper right", prop={"size":14})
    plt.grid(which="both")
    
    plt.tight_layout()
    plt.show()
    
neighbor_means = []
neighbor_medians = []
for UP_TO in tqdm(range(1, BLEND_NEIGHBORS), total=BLEND_NEIGHBORS-1):
    mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
    neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
    
    median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
    neighbor_medians.append(np.abs(gt_breath_pressure-median_pressure).mean())
    
    
    
# ERROR PLOT - WHICH NUMBER OF NEIGHBORS IS BEST
plt.figure(figsize=(20, 8))

MIN_MAE = min(min(neighbor_means), min(neighbor_medians))

plt.plot(neighbor_means, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_means, y2=MIN_MAE, color="deepskyblue", alpha=0.25)

plt.plot(neighbor_medians, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
plt.fill_between(x=np.arange(BLEND_NEIGHBORS-1), y1=neighbor_medians, y2=MIN_MAE, color="mediumspringgreen", alpha=0.25)

IDEAL_NUMBER_OF_NEIGHBORS = np.argmin(neighbor_medians) if min(neighbor_means)>min(neighbor_medians) else np.argmin(neighbor_means)
BLEND_STYLE = "MEDIAN" if min(neighbor_means)>min(neighbor_medians) else "MEAN"
plt.axvline(x=IDEAL_NUMBER_OF_NEIGHBORS, color='blue', linestyle="dotted")

plt.xlabel("Number of Neighbors", fontweight="bold")
plt.ylabel("MAE When Compared to Ground Truth", fontweight="bold")
plt.title(f"NEAREST NEIGHBOR MEAN v. MEDIAN FOR DIFFERENT AMOUNTS OF NEIGHBORS", fontweight="bold")
plt.legend(loc="upper right", prop={"size":14})
plt.grid(which="both")

plt.tight_layout()
plt.show()

print(f"\n\n... THE IDEAL NUMBER OF NEIGHBORS IS {IDEAL_NUMBER_OF_NEIGHBORS}, USING `{BLEND_STYLE}` WITH AN MAE OF {MIN_MAE} ...\n\n")

In [ ]:
MAX_DISTANCE = 5000
IDEAL_NEIGHBOR_MAX = 25
CALCULATE_WEIGHTED = True

global_nn_means, global_nn_medians = [], []
global_wtd_nn_means, global_wtd_nn_medians = [], []

for BREATH_ID in tqdm(sorted(train_df.breath_id.values.get())[:20], total=20,):
    gt_breath = train_df.loc[train_df.breath_id==BREATH_ID].to_pandas()
    BREATH_R = gt_breath.R.values[0]
    BREATH_C = gt_breath.C.values[0]
    df_to_use = train_df.copy()[(train_df.R==BREATH_R)&(train_df.C==BREATH_C)].reset_index(drop=True) if RESTRICT_RC else train_df.copy()

    # Fit the model
    model = NearestNeighbors(n_neighbors=IDEAL_NEIGHBOR_MAX, metric="l1",)
    model.fit(df_to_use[USE_COLS])

    distances, indices = model.kneighbors(gt_breath[USE_COLS])
    distances, indices = distances[0, 1:], indices[0, 1:] #discard GT

    gt_breath_pressure = gt_breath[PRESSURE_COLS].squeeze().values
    nn_pressure_df = df_to_use.iloc[indices][PRESSURE_COLS]
    nn_pressure_df.insert(loc=0, name="nn_distance", value=distances)
    nn_pressures = nn_pressure_df[PRESSURE_COLS].values

    neighbor_means, neighbor_medians = [], []
    wtd_neighbor_means, wtd_neighbor_medians = [], []
    
    for UP_TO in range(1, IDEAL_NEIGHBOR_MAX):
        
        # Calculate Unweighted
        mean_pressure = nn_pressures[:UP_TO].mean(axis=0).get()
        median_pressure = np.median(nn_pressures[:UP_TO].get(),axis=0)
        neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
        neighbor_medians.append(np.abs(gt_breath_pressure-median_pressure).mean())
        
        if CALCULATE_WEIGHTED:
            wtd_mean_pressure = np.average(nn_pressures[:UP_TO], weights=MAX_DISTANCE-np.clip(distances[:UP_TO], 0, MAX_DISTANCE), axis=0).get()
            wtd_median_pressure = np.array([compute_weighted_median(_values=nn_pressures[:UP_TO, i].get(), _weights=(MAX_DISTANCE-np.clip(distances[:UP_TO], 0, MAX_DISTANCE))**0.5, redux_factor=3) for i in range(80)])
            
            wtd_neighbor_means.append(np.abs(gt_breath_pressure-mean_pressure).mean())
            wtd_neighbor_medians.append(np.abs(gt_breath_pressure-wtd_median_pressure).mean())
            
    global_nn_means.append(neighbor_means)
    global_nn_medians.append(neighbor_medians)
    
    if CALCULATE_WEIGHTED:
        global_wtd_nn_means.append(wtd_neighbor_means)
        global_wtd_nn_medians.append(wtd_neighbor_medians)
    
global_nn_mean_reduced = np.array(global_nn_means).mean(axis=0)
global_nn_median_reduced = np.array(global_nn_medians).mean(axis=0)

if CALCULATE_WEIGHTED:
    global_wtd_nn_mean_reduced = np.array(global_wtd_nn_means).mean(axis=0)
    global_wtd_nn_median_reduced = np.array(global_wtd_nn_medians).mean(axis=0)

# ERROR PLOT - WHICH NUMBER OF NEIGHBORS IS BEST
plt.figure(figsize=(20, 8))

plt.plot(global_nn_mean_reduced, color="deepskyblue", linewidth=3, label="NEIGHBOR MEAN")
plt.plot(global_nn_median_reduced, color="mediumspringgreen", linewidth=3, label="NEIGHBOR MEDIAN")
plt.fill_between(x=np.arange(IDEAL_NEIGHBOR_MAX-1), y1=global_nn_mean_reduced, y2=global_nn_median_reduced, color="lightblue", alpha=0.5)

if CALCULATE_WEIGHTED:
    plt.plot(global_wtd_nn_mean_reduced, color="orangered", linewidth=3, label="NEIGHBOR WTD MEAN")
    plt.plot(global_wtd_nn_median_reduced, color="hotpink", linewidth=3, label="NEIGHBOR WTD MEDIAN")
    plt.fill_between(x=np.arange(IDEAL_NEIGHBOR_MAX-1), y1=global_wtd_nn_mean_reduced, y2=global_wtd_nn_median_reduced, color="lightcoral", alpha=0.5)

plt.xticks(np.arange(IDEAL_NEIGHBOR_MAX-1))
plt.xlabel("Number of Neighbors", fontweight="bold")
plt.ylabel("MAE Averaged Across First 100 Training Breaths", fontweight="bold")
plt.title(f"NEAREST NEIGHBOR MEAN v. MEDIAN FOR DIFFERENT AMOUNTS OF NEIGHBORS", fontweight="bold")
plt.legend(loc="upper right", prop={"size":14})
plt.grid(which="both")

plt.tight_layout()
plt.show()

**Try by weighting the neighbors using supplied distance**

# Make Nearest Neighbor Embedding for All Train And Test Examples

In [ ]:
# [1,2,3,4,5] # median=3
#
# values = [1,2,3,4,5]
# weights = [2,2,2,1,1]
# w_values = [1,1,2,2,3,3,4,5]

In [ ]:
NN_TO_USE = 10
breath_ids, mean_test_pressures, median_test_pressures = [], [], []
for _R in test_df.R.unique().to_pandas():
    for _C in test_df.C.unique().to_pandas():
        test_df_to_use = test_df[(test_df.R==_R)&(test_df.C==_C)].reset_index(drop=True)
        train_df_to_use = train_df[(train_df.R==_R)&(train_df.C==_C)].reset_index(drop=True)
        model = NearestNeighbors(n_neighbors=NN_TO_USE, metric="l1")
        model.fit(train_df_to_use[USE_COLS])
        distances, indices = model.kneighbors(test_df_to_use[USE_COLS])
        distances, indices = distances.values, indices.values
        for i, BREATH_ID in tqdm(enumerate(sorted(test_df_to_use.breath_id.values.get())), total=len(test_df_to_use)):
            breath_ids.append(BREATH_ID)
            _distances, _indices = distances[i], indices[i]
            nn_pressures = train_df_to_use.iloc[_indices][PRESSURE_COLS].values.get()         
            mean_test_pressures.append(np.average(nn_pressures, axis=0, weights=_distances.get()))
            median_test_pressures.append(np.median(nn_pressures,axis=0))

In [ ]:
ss_df = pd.merge(left=ss_df, right=pd.read_csv("../input/ventilator-pressure-prediction/test.csv")[["id", "breath_id"]], on="id")
ss_df["median_pressure"] = ss_df["pressure"].copy()
ss_df["mean_pressure"] = ss_df["pressure"].copy()

sort_indices = np.argsort(breath_ids)
ss_df["mean_pressure"] = np.array(mean_test_pressures)[sort_indices].reshape(-1)
ss_df["median_pressure"] = np.array(median_test_pressures)[sort_indices].reshape(-1)
ss_df["pressure"] = (ss_df["mean_pressure"]+ss_df["median_pressure"])/2
ss_df

In [ ]:
for x in ["median", "mean", None]:
    p_name = "pressure" if not x else f"{x}_pressure"
    tmp_csv_df = ss_df[["id", p_name]]
    tmp_csv_df.columns = ["id", "pressure"]
    tmp_csv_df.to_csv(f"{p_name}_submission.csv", index=False)

# Make Nearest Neighbor Embedding for All Train And Test Examples

In [ ]:
# # BLEND_NEIGHBORS = len(train_df) --> WILL GO OOM
BLEND_NEIGHBORS = 1000
CLIP_DISTANCE_MIN = 0.0
CLIP_DISTANCE_MAX = 1500.0
DISTANCE_MATRIX = np.zeros((len(train_df), len(train_df)), dtype=np.uint8)

model = NearestNeighbors(n_neighbors=BLEND_NEIGHBORS, metric="l1",)
model.fit(train_df[USE_COLS])
all_distances, all_indices = model.kneighbors(train_df[USE_COLS])

for distances, indices in tqdm(zip(all_distances.values, all_indices.values), total=len(train_df)):
    distances, indices = cupy.clip(distances[1:], CLIP_DISTANCE_MIN, CLIP_DISTANCE_MAX), indices[1:]
    DISTANCE_MATRIX[i, indices] = (255*(1-(distances-CLIP_DISTANCE_MIN)/(CLIP_DISTANCE_MAX))).astype(np.uint8)

In [ ]:
# np.savez("nxn_distance_matrix.npz", DISTANCE_MATRIX)
print(f"TRAIN DATAFRAME LENGTH : {len(train_df)}")
print(f"DISTANCE MATRIX SHAPE  : {DISTANCE_MATRIX.shape}")